In [ ]:
# DDGR20，嵌入近似hint，l-noisy < <v,s> < l+noisy
cd framework

In [ ]:
load("../framework/instance_gen.sage")
import numpy as np
import random

In [ ]:
n = 128
m = n
q = 3329
D_s = build_centered_binomial_law(3)
D_e = D_s
A, b, s, dbdd = initialize_from_LWE_instance(DBDD_predict, n, q, m, D_e, D_s)
print(s)
beta, delta = dbdd.estimate_attack()
_ = dbdd.integrate_q_vectors(q, report_every=20)

In [ ]:
# 使用拒绝采样生成[-3*sigma,3*sigma]之间满足高斯分布的随机数
def sample_discrete_gaussian(sigma):
    while True:
        x = random.randint(int(-3 * sigma), int(3 * sigma)) 
        acceptance_prob = math.exp(-x**2 / (2 * sigma**2)) 
        if random.uniform(0, 1) < acceptance_prob: 
            return x
        
def generate_se_eta_sigma_approx_hint(m, n, q, sigma, k):
  V = []
  L = []

  for i in range(k):
    D_e = {-3: 1/64, -2: 6/64, -1: 15/64, 0: 20/64, 1: 15/64, 2: 6/64, 3: 1/64}
    values, probabilities = zip(*D_e.items())
    v = np.array(np.random.choice(values, size=m+n, p=probabilities))
    e = sample_discrete_gaussian(sigma)
    l = dbdd.leak(v) + e
    V.append(v)
    L.append(l)
  print("L",L)
  return V,L

In [ ]:
num_hint = 300
sigma = 2
V, L = generate_se_eta_sigma_approx_hint(m, n, q, sigma, num_hint)
for j in range(num_hint):
    print("the ",j+1,"-th secret error approx hint")
    _ = dbdd.integrate_approx_hint(vec(V[j]), L[j], sigma**2, aposteriori=False)
    _ = dbdd.integrate_q_vectors(q, report_every=20)